Please set an output directory to store model and other artifacts

In [ ]:
#!pip install mne
#!pip install accelerate -U
#!unzip test_code.zip -d test_code
#!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 5.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinu

In [ ]:
#automatically creates a config file for training
from accelerate.utils import write_basic_config
write_basic_config()

PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

In [ ]:
#session needs to be restarted after calling the above
import os
os._exit(00)

In [ ]:
#!mkdir output

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
def main(num_epochs, bs, seed):

    import os
    from rich import print
    import mne
    import numpy as np
    import torch
    import torch.nn as nn
    from accelerate.utils import tqdm
    from accelerate.utils import set_seed
    from torch.utils.data import DataLoader
    from accelerate import DistributedDataParallelKwargs
    import wandb

    from transformers import Wav2Vec2Config, Wav2Vec2ForPreTraining
    from accelerate import Accelerator
    from transformers.models.wav2vec2.modeling_wav2vec2 import (
        ACT2FN,
        Wav2Vec2FeatureEncoder,
        _compute_mask_indices,
        _sample_negative_indices,
        Wav2Vec2GroupNormConvLayer,
        Wav2Vec2GumbelVectorQuantizer,
        Wav2Vec2LayerNormConvLayer,
        Wav2Vec2NoLayerNormConvLayer
    )

    # Initializing a Wav2Vec2 facebook/wav2vec2-base-960h style configuration
    configuration = Wav2Vec2Config.from_pretrained("facebook/wav2vec2-large")
    model = Wav2Vec2ForPreTraining.from_pretrained("facebook/wav2vec2-large", config=configuration)

    configuration.conv_kernel = [3, 2, 2, 2, 2, 2]
    configuration.conv_stride = [3, 2, 2, 2, 2, 2]
    configuration.conv_dim = [512, 512, 512, 512, 512, 512]
    configuration.num_feat_extract_layers = 6
    configuration.num_channels = 20
    configuration.num_negatives = 20
    configuration.mask_time_prob = 0.065
    configuration.mask_time_length = 10

    # configuration.diversity_loss_weight = 0.0

    # set some training arguments here
    # save steps and log steps control saving and printing output respectively
    save_steps = 10
    logging_steps = 1
    learning_rate = 1e-2
    weight_decay = 0.01
    warmup_ratio = 0.05

    remove_quantization = False  # set to true to remove quantization
    # put output directory for models here
    output_dir = "output"
    # "/data/work/zeydabadi/test/"
    processed_folder = "/content/test_code"

    class Wav2Vec2LayerNormConvLayer(Wav2Vec2LayerNormConvLayer):
        def __init__(self, config, layer_id=0):
            super().__init__(config, layer_id)

            self.in_conv_dim = config.conv_dim[layer_id -
                                               1] if layer_id > 0 else config.num_channels
            self.out_conv_dim = config.conv_dim[layer_id]

            self.conv = nn.Conv1d(
                self.in_conv_dim,
                self.out_conv_dim,
                kernel_size=config.conv_kernel[layer_id],
                stride=config.conv_stride[layer_id],
                bias=config.conv_bias,
                padding=config.conv_kernel[layer_id]//2,
            )
            self.activation = ACT2FN[config.feat_extract_activation]
            self.layer_norm = nn.LayerNorm(
                self.out_conv_dim, elementwise_affine=True)

    class Wav2Vec2NoLayerNormConvLayer(Wav2Vec2NoLayerNormConvLayer):
        def __init__(self, config, layer_id=0):
            super().__init__(config, layer_id)

            self.in_conv_dim = config.conv_dim[layer_id -
                                               1] if layer_id > 0 else config.num_channels
            self.out_conv_dim = config.conv_dim[layer_id]

            self.conv = nn.Conv1d(
                self.in_conv_dim,
                self.out_conv_dim,
                kernel_size=config.conv_kernel[layer_id],
                stride=config.conv_stride[layer_id],
                bias=config.conv_bias,
                padding=config.conv_kernel[layer_id]//2,
            )
            self.activation = ACT2FN[config.feat_extract_activation]

    class Wav2Vec2GroupNormConvLayer(Wav2Vec2GroupNormConvLayer):
        def __init__(self, config, layer_id=0):
            super().__init__(config, layer_id)

            self.in_conv_dim = config.conv_dim[layer_id -
                                               1] if layer_id > 0 else config.num_channels
            self.out_conv_dim = config.conv_dim[layer_id]

            self.conv = nn.Conv1d(
                self.in_conv_dim,
                self.out_conv_dim,
                kernel_size=config.conv_kernel[layer_id],
                stride=config.conv_stride[layer_id],
                bias=config.conv_bias,
                padding=config.conv_kernel[layer_id]//2,
            )

            self.activation = ACT2FN[config.feat_extract_activation]
            self.layer_norm = nn.GroupNorm(
                num_groups=self.out_conv_dim, num_channels=self.out_conv_dim, affine=True)

    class Wav2Vec2FeatureEncoder(Wav2Vec2FeatureEncoder):

        def __init__(self, config):
            super().__init__(config)

            if config.feat_extract_norm == "group":
                conv_layers = [Wav2Vec2GroupNormConvLayer(config, layer_id=0)] + [
                    Wav2Vec2NoLayerNormConvLayer(config, layer_id=i + 1) for i in range(config.num_feat_extract_layers - 1)
                ]
            elif config.feat_extract_norm == "layer":
                conv_layers = [
                    Wav2Vec2LayerNormConvLayer(config, layer_id=i) for i in range(config.num_feat_extract_layers)
                ]
            else:
                raise ValueError(
                    f"`config.feat_extract_norm` is {config.feat_extract_norm}, but has to be one of ['group', 'layer']"
                )

            self.conv_layers = nn.ModuleList(conv_layers)
            self.gradient_checkpointing = False
            self._requires_grad = True

        def forward(self, input_values):
            hidden_states = input_values

            # make sure hidden_states require grad for gradient_checkpointing
            if self._requires_grad and self.training:
                hidden_states.requires_grad = True

            for conv_layer in self.conv_layers:
                if self._requires_grad and self.gradient_checkpointing and self.training:
                    hidden_states = self._gradient_checkpointing_func(
                        conv_layer.__call__,
                        hidden_states,
                    )
                else:
                    hidden_states = conv_layer(hidden_states)

            return hidden_states

    class Wav2Vec2NoQuantizer(Wav2Vec2GumbelVectorQuantizer):
        def __init__(self, config):
            super().__init__(config)
            self.proj = nn.Linear(config.conv_dim[-1], config.codevector_dim)

        @staticmethod
        def _compute_perplexity(probs, mask=None):
            # should probably make this a tensor
            return 0

        def forward(self, hidden_states, mask_time_indices=None):
            hidden_states = self.proj(hidden_states)
            perplexity = self._compute_perplexity(
                hidden_states,  mask_time_indices)

            return hidden_states, perplexity

    model.wav2vec2.feature_extractor = Wav2Vec2FeatureEncoder(configuration)

    if remove_quantization:
        model.wav2vec2.quantizer = Wav2Vec2NoQuantizer(configuration)
        model.config.diversity_loss_weight = 0.0

    def list_files_with_extension(root_folder, extension):
        """
        List all files with a specific extension in a folder and its subfolders.

        Parameters:
        - root_folder (str): The root folder to start the search.
        - extension (str): The file extension to look for (e.g., '.txt').

        Returns:
        - List of file paths with the specified extension.
        """
        matching_files = []
        walk_list = list(os.walk(root_folder))
        sorted_walk_list = sorted(walk_list, key=lambda x: x[0])

        # Traverse the directory tree
        for dirpath, dirnames, filenames in sorted_walk_list:
            for filename in filenames:
                if filename.endswith(extension):
                    full_path = os.path.join(dirpath, filename)
                    matching_files.append(full_path)

        return matching_files

    # im using dataset class from shared file
    class EEGDataset(torch.utils.data.Dataset):
        def __init__(self, path, extension):
            self.path = path
            self.extension = extension
            self.items = list_files_with_extension(self.path, self.extension)
            # self.items = self.filter_valid_files(all_items)

        def __len__(self):
            return len(self.items)

        def get_filename(self, idx):
            return self.items[idx]

        def __getitem__(self, idx):
            file_id = self.items[idx]
            raw_file = mne.io.read_raw_fif(file_id, preload=True)
            data = raw_file.get_data()
            # model expects dictionary with at least this keyword
            return {"input_values": torch.Tensor(data)}

    mne.set_log_level("ERROR")

    train_dataset = EEGDataset(processed_folder, ".fif")

    ep = train_dataset[0]
    num_samples = ep["input_values"].shape[1]

    for i, s in enumerate(configuration.conv_stride):
        if s > 1:
            num_samples = (num_samples + 2*(configuration.conv_kernel[i]//2) - (
                configuration.conv_kernel[i]-1) - 1)//s + 1

    set_seed(seed)

    accelerator = Accelerator(kwargs_handlers=[DistributedDataParallelKwargs(find_unused_parameters=True)])
    # accelerator.print("Started")
    if accelerator.is_main_process:
      run = wandb.init(
          project = "multigpu_test_kaggle",
          config = {
              "lr": learning_rate,
              "bs": bs,
              "num_epochs": num_epochs
          }
      )
    accelerator.wait_for_everyone()

    train_dataloader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
    # progress_bar = tqdm(train_dataloader, disable=not accelerator.is_main_process)

    optimizer = torch.optim.Adam(
        params=model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=learning_rate, epochs=num_epochs, steps_per_epoch=len(train_dataloader), pct_start=warmup_ratio)

    model, optimizer, train_dataloader, scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, scheduler)
    # accelerator.print("prepare done")
    steps = 0
    for epoch in range(num_epochs):
        model.train()
        for batch in tqdm(train_dataloader):
            steps += 1
            bs_data, cin, seq_length = batch["input_values"].shape
            seq_length = int(num_samples)
            mask_time_indices = _compute_mask_indices(shape=(
                bs_data, seq_length), mask_prob=configuration.mask_time_prob, mask_length=configuration.mask_time_length)
            sampled_negative_indices = _sample_negative_indices(features_shape=(
                bs_data,  seq_length), num_negatives=configuration.num_negatives, mask_time_indices=mask_time_indices)

            mask_time_indices = torch.tensor(
                data=mask_time_indices, device=accelerator.device, dtype=torch.long)
            sampled_negative_indices = torch.tensor(
                data=sampled_negative_indices, device=accelerator.device, dtype=torch.long)

            # accelerator.print("before model forward ")

            outputs = model(input_values=batch["input_values"], mask_time_indices=mask_time_indices,
                            sampled_negative_indices=sampled_negative_indices, return_dict=True)
            loss = outputs.loss

            # accelerator.print("shape of loss ", loss)
            # accelerator.print("loss mean ", loss.mean())
            gathered_loss = accelerator.gather(loss)
            # accelerator.print("shape of gathered loss ", gathered_loss)

            if accelerator.is_main_process:
                wandb.log({"loss": gathered_loss.mean().item()})

            if steps % logging_steps == 0:
                accelerator.print(
                    f"Doing step {steps}, loss: %.2f" % (gathered_loss.mean()))
            if steps % save_steps == 0:
                accelerator.wait_for_everyone()
                unwrapped_model = accelerator.unwrap_model(model)
                accelerator.save_model(
                    unwrapped_model, f"{output_dir}/model_{steps}")
                del unwrapped_model

            accelerator.backward(loss)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
    if accelerator.is_main_process:
      wandb.finish()

In [ ]:
from accelerate import notebook_launcher

In [ ]:
#Pass number of epochs, batch size and seed here. In order.
args = (5, 2, 42)

In [ ]:
notebook_launcher(main, args, num_processes = 1)

Launching training on one GPU.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large were not used when initializing Wav2Vec2ForPreTraining: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForPreTraining from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForPreTraining from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForPreTraining were not initialized from the model checkpoint at facebook/wav2vec2-large and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should pr

...

  0%|          | 0/3 [00:00<?, ?it/s]

Doing step 10, loss: 61.76
Doing step 11, loss: 62.15
Doing step 12, loss: 31.03


  0%|          | 0/3 [00:00<?, ?it/s]

Doing step 13, loss: 59.91
Doing step 14, loss: 62.44
Doing step 15, loss: 31.75


loss,▆▆▂█▃▁▅▅▂▅▅▂▅▅▂
loss,31.75071
